In [ ]:
import requests
from PIL import Image,ImageEnhance
from io import BytesIO
import numpy as np
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
import pandas as pd

import requests
from PIL import Image
from io import BytesIO
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing import image


def adjust_contrast_brightness(img, contrast_factor=1.2, brightness_factor=1.2):

    enhancer = ImageEnhance.Contrast(img)
    img = enhancer.enhance(contrast_factor)

    enhancer = ImageEnhance.Brightness(img)
    img = enhancer.enhance(brightness_factor)

    return img

def load_image_from_url(url, target_size=(224, 224), contrast_factor=1.2, brightness_factor=1.2):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes
        img = Image.open(BytesIO(response.content))

        # Adjust contrast and brightness
        img = adjust_contrast_brightness(img, contrast_factor, brightness_factor)

        img = img.resize(target_size)
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        img = preprocess_input(img)
        return img
    except Exception as e:
        print(f"Error loading image from URL {url}: {e}")
        return None


def extract_features_from_urls(image_urls):
    features_list = []

    for urls in image_urls:
        best_feature = None
        best_score = float('-inf')

        for url in urls:  # Assuming URLs are stored as a string representation of a list
            img = load_image_from_url(url)
            img_features = model.predict(img)

            # Compare current feature score with the best score
            if np.sum(img_features) > best_score:
                best_score = np.sum(img_features)
                best_feature = img_features

        features_list.append(best_feature)

    return features_list



model = VGG16(weights='imagenet', include_top=False, pooling='avg')
    # Read CSV file containing image URLs and text reviews
df = pd.read_csv("A2_Data.csv")

# Extract image URLs from the 'image' column
image_urls_list = df['Image'].apply(eval).tolist()

# Extract features from images
extracted_features = extract_features_from_urls(image_urls_list)

# Convert the list of feature vectors into a numpy array
feature_matrix = np.array(extracted_features)

# Normalize the feature matrix
normalized_features = (feature_matrix - np.mean(feature_matrix)) / np.std(feature_matrix)


In [ ]:
normalized_features_list = []

# Iterate through each sublist in normalized_features_list
for sublist in normalized_features:
    # Iterate through each element in the sublist
    for item in sublist:
        # Append the element to the single list
        normalized_features_list.append(item)

print(normalized_features_list[993])

In [10]:
import pickle
import numpy as np
import pandas as pd


# Store as pickle
with open('normalized_features_list_latest.pkl', 'wb') as f:
    pickle.dump(normalized_features_list, f)

# Load the pickle file
with open('normalized_features_list_latest.pkl', 'rb') as f:
    loaded_normalized_features_list = pickle.load(f)

In [ ]:
df_new_tst= pd.DataFrame({'vector_column': loaded_normalized_features_list})


df_latest = pd.read_csv("A2_Data.csv")

concatenated_df = pd.concat([df_latest, df_new_tst], axis=1)

# Reset index of the concatenated DataFrame
concatenated_df.reset_index(drop=True, inplace=True)

concatenated_df

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.metrics.pairwise import cosine_similarity
import string
import math

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

custom_punctuation = string.punctuation + '“”‘’'
tokenizer = nltk.RegexpTokenizer(r"\w+")
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def preprocessing_text_mod(text):
    if pd.isna(text):
        return ''  # Return an empty string for NaN values
    text = str(text)  # Convert to string to handle non-string values
    text = text.lower()  # Lowercase the text
    tokens = tokenizer.tokenize(text)  # Tokenization
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    tokens = [token for token in tokens if token not in custom_punctuation]  # Remove punctuations
    tokens = [token for token in tokens if token.strip() != '']


    # Stemming
    tokens = [stemmer.stem(token) for token in tokens]
    # Lemmatization
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    return ' '.join(tokens)



# Preprocess the content
concatenated_df['Preprocessed_Content'] = concatenated_df['Review Text'].apply(preprocessing_text_mod)

# Calculate TF-IDF scores
def calculate_tf(word_list):
    word_tf = {}
    total_words = len(word_list)
    for word in word_list:
        word_tf[word] = word_tf.get(word, 0) + 1 / total_words
    return word_tf

def calculate_idf(documents):
    word_idf = {}
    total_documents = len(documents)
    all_words = set([word for sublist in documents for word in sublist])
    for word in all_words:
        count = sum([1 for document in documents if word in document])
        word_idf[word] = math.log(total_documents / count)
    return word_idf

def calculate_tf_idf(tf, idf):
    tf_idf = {}
    for word, tf_value in tf.items():
        tf_idf[word] = tf_value * idf[word]
    return tf_idf

documents = [word_tokenize(doc) for doc in concatenated_df['Preprocessed_Content']]

word_tf_list = [calculate_tf(doc) for doc in documents]

word_idf = calculate_idf(documents)

tf_idf_list = [calculate_tf_idf(tf, word_idf) for tf in word_tf_list]



#Convert tf-idf to DataFrame
#tf_idf_df = pd.DataFrame(tf_idf_list)
#df1 = pd.concat([df, tf_idf_df], axis=1)

#df1


In [63]:


def cosine_similarity_fresh(vec1, vec2):

    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)

    if norm_vec1 != 0 and norm_vec2 != 0:
        similarity_fresh = dot_product / (norm_vec1 * norm_vec2)
    else:
        similarity_fresh = 0  # If any of the vectors is a zero vector, cosine similarity is 0.

    return similarity_fresh


# Calculate TF-IDF scores for the input text review
def calculate_similarity(input_review):
    preprocessed_input_review = preprocessing_text_mod(input_review)
    input_review_tokens = word_tokenize(preprocessed_input_review)
    input_review_tf = calculate_tf(input_review_tokens)
    input_review_tf_idf = calculate_tf_idf(input_review_tf, word_idf)

    # Represent the input text review as a vector
    input_review_vector = [input_review_tf_idf.get(word, 0) for word in word_idf.keys()]

    # Compute cosine similarity between input review vector and each review vector in DataFrame
    similarities1 = []
    for tf_idf in tf_idf_list:
        review_vector = [tf_idf.get(word, 0) for word in word_idf.keys()]
        similarity = cosine_similarity_fresh(input_review_vector, review_vector)
        similarities1.append(similarity)

    return similarities1

# Add similarity scores to DataFrame



def takingReview(review_input):

  similarities = calculate_similarity(review_input)
  concatenated_df['Cosine_Similarity_text'] = similarities






In [64]:
def calculate_cosine_similarity(input_features, target_features):
    similarities = cosine_similarity(input_features, target_features)
    return similarities.flatten()


def calculate_image_similarity(user_url):
    user_features = load_image_from_url(user_url)
    if user_features is not None:
        user_features = model.predict(user_features)
        user_normalized_features = (user_features - np.mean(user_features)) / np.std(user_features)

        # Calculate cosine similarities with each image in the DataFrame column
        similarities_list1 = []
        for features in concatenated_df['vector_column']:
            similarity = calculate_cosine_similarity(user_normalized_features, np.array(features).reshape(1, -1))
            similarities_list1.append(similarity[0])

        return similarities_list1
    else:
        return None



def takingUrl(url_input):

  similarities_list = calculate_image_similarity(url_input)

  concatenated_df['Cosine_Similarity_images'] = similarities_list









In [ ]:
input_txt=input("Enter input review")#"guitar is very"
input_img_url=input("enter your input url")#"https://images-na.ssl-images-amazon.com/images/I/71Isri9SEaL._SY88.jpg"

In [ ]:
takingReview(input_txt)





takingUrl(input_img_url)

In [ ]:
concatenated_df['average'] = (concatenated_df['Cosine_Similarity_text'] + concatenated_df['Cosine_Similarity_images']) / 2
concatenated_df

In [ ]:
result_df = concatenated_df.sort_values(by='Cosine_Similarity_images', ascending=False)

result_df

In [ ]:
df_sorted_on_text = concatenated_df.sort_values(by='Cosine_Similarity_text', ascending=False)
df_sorted_on_text

In [ ]:
def print_dataframe_info(dataframe):
    for index, row in dataframe.head(3).iterrows():
        print("Image URL:", row['Image'])
        print("Review:", row['Review Text'])
        print("Cosine similarity of images -", '{:.4f}'.format(row['Cosine_Similarity_images']))
        print("Cosine similarity of text -", '{:.3f}'.format(row['Cosine_Similarity_text']))
        print("Composite score -", '{:.3f}'.format(row['average']))
        print()
    # subset1 = dataframe['Cosine_Similarity_text'].head(3)
    # # Calculate the average of the subset
    # avg1 = subset1.mean()
    # subset2 = dataframe['Cosine_Similarity_images'].head(3)
    # # Calculate the average of the subset
    # avg2 = subset2.mean()
    # total_avg=(avg1+avg2)/2

    # print("Composite similarity scores of images: {:.4f}".format(avg2))
    # print("Composite similarity scores of text: {:.4f}".format(avg1))
    # print("Final composite similarity score: {:.4f}".format(total_avg))
    print("\n")


# Assuming `df` is your DataFrame with columns 'Image URL', 'Review', 'Cosine similarity of images', 'Cosine similarity of text'
print("Image based retrival")
print_dataframe_info(result_df)

print("Text based retrival")
print_dataframe_info(df_sorted_on_text)

In [ ]:
df_sorted_on_composite=concatenated_df.sort_values(by='average', ascending=False)
df_sorted_on_composite

In [ ]:
print("composite score based retrival")
print_dataframe_info(df_sorted_on_composite)